In [ ]:
import os
gpu_num = "" # Use "" to use the CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

resolution = [480,320] # increase for higher quality of renderings

# Allows to exit cell execution in Jupyter
class ExitCell(Exception):
    def _render_traceback_(self):
        pass

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

tf.random.set_seed(1) # Set global random seed for reproducibility

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import time

# Import Sionna RT components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, RadioMaterial, LambertianPattern
from sionna.constants import PI

# For link-level simulations
from sionna.channel import cir_to_ofdm_channel, subcarrier_frequencies, OFDMChannel, ApplyOFDMChannel, CIRDataset
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver
from sionna.utils import compute_ber, ebnodb2no, PlotBER
from sionna.ofdm import KBestDetector, LinearDetector
from sionna.mimo import StreamManagement

In [ ]:
# Load integrated scene
scene = load_scene("../scenes/accenture_5G_indoor/Acccenture_inside_building.xml")

In [ ]:
scene.get("elm__4").radio_material = "itu_glass"
scene.get("elm__5").radio_material = "itu_brick"
scene.get("elm__6").radio_material = "itu_concrete"

In [ ]:
epsilon_r_complex = scene.get("elm__4").radio_material.complex_relative_permittivity

In [ ]:
print(epsilon_r_complex)
k = np.imag(np.sqrt(epsilon_r_complex))
n = np.real(np.sqrt(epsilon_r_complex))
n,k

In [ ]:
R = ((1-n)**2+k**2)/((1+n)**2+k**2)
R

In [ ]:
xy_center = [98,80]
z_centers = [5.6, 1.5]

In [ ]:
sampling_frequency = 10e6
sample_m = 62.0 / sampling_frequency

c0 = sionna.SPEED_OF_LIGHT

fc = 3.6e9
scene.frequency = fc # in Hz; implicitly updates RadioMaterials

scene.synthetic_array = False # If set to False, ray tracing will be done per antenna element (slower for large arrays)

In [ ]:
epsilon_0 = 8.854 * 10**-12
T = 1-R
alpha = (2 * 2 * np.pi * fc * np.abs(k))/c0
print(T, alpha)
T * (1 - np.e ** -(alpha*0.03)) * T

In [ ]:
da = 0.5
M = 8
N = 4

scene._clear()

# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=N,
                             num_cols=M,
                             vertical_spacing=da,
                             horizontal_spacing=da,
                             pattern="tr38901",
                             polarization="V")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=da,
                             horizontal_spacing=da,
                             pattern="tr38901",
                             polarization="V")
                             #polarization="cross")

In [ ]:
transmitters = [
    {
        "name":"tx_outdoor_1",
        "position":[95.4,114.6,18.11],
        "orientation":[0,0,0],
        "antenna_type":"dipole"
    },
    {
        "name":"tx_outdoor_2",
        "position":[101,114.6,18.11],
        "orientation":[0,0,0],
        "antenna_type":"dipole"
    },
    {
        "name":"tx_indoor_1",
        "position":[94,60,13.41],
        "orientation":[0,0,0],
        "antenna_type":"dipole"
    },
]

receivers = [
    {
        "name":"rx_outdoor_1",
        "position":[43.9394, 125.376, 1.5+5.56987],
        "orientation":[0,0,0]
    },
    {
        "name":"rx_outdoor_2",
        "position":[153, 125.376, 1.5+5.56987],
        "orientation":[0,0,0]
    },
    {
        "name":"rx_indoor_1",
        "position":[94,20, 1.5],
        "orientation":[0,0,0]
    },
]

In [ ]:
for i in range(len(transmitters)):
    tx = Transmitter(name=transmitters[i]["name"],
                 position=transmitters[i]["position"])
    rx = Receiver(name=receivers[i]["name"],
              position=receivers[i]["position"])
    scene.add(tx)
    scene.add(rx)
    tx.look_at(rx)
    rx.look_at(tx)

In [ ]:
cm_level_1 = scene.coverage_map(num_samples=10e5,
                            max_depth=5,
                            diffraction=True,
                            cm_center=xy_center + [z_centers[0]],
                            cm_orientation=[0,0,0],
                            cm_size=[400,400],
                            cm_cell_size=[1,1])

cm_level_2 = scene.coverage_map(num_samples=10e5,
                            max_depth=5,
                            diffraction=True,
                            cm_center=xy_center + [z_centers[1]],
                            cm_orientation=[0,0,0],
                            cm_size=[400,400],
                            cm_cell_size=[1,1])

In [ ]:
cm_level_1.show(tx=0);
cm_level_1.show(tx=1);
cm_level_1.show(tx=2);

In [ ]:
cm_level_2.show(tx=0);
cm_level_2.show(tx=1);
cm_level_2.show(tx=2);

In [ ]:
scene.preview(coverage_map=cm_level_2, cm_tx=2)